# Bedrock with LangChain using a Prompt that includes Context

please refer the detail : https://github.com/aws-samples/amazon-bedrock-workshop/blob/main/01_Generation/02_contextual_generation.ipynb

## install dependencies

In [122]:
# dependencies
!pip install --upgrade boto3

In [123]:
import json
import os
import sys

import boto3
import botocore

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww

boto3_bedrock = bedrock.get_bedrock_client()

Create new client
  Using region: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)


In [124]:

from langchain.prompts import PromptTemplate

template = """你是一个资深的游戏数据分析师, 目前你在对一款FPS类型游戏进行数据分析, 会根据<requirements>来输出结果。
KDA的计算公式为 （击杀+助攻）/死亡

<reference>
玩家姓名: 小明
玩家设备评分：设备评分90，iPhone11性能良好，能够流畅运行游戏
大厅网络评分：大厅网络评分80，平均延迟在可接受范围内
战斗网络评分：战斗网络评分90，平均延迟极佳
战斗帧率评分：帧率评分85，iPhone11在当前图形设置下能够保持较高且稳定的帧率
战斗KDA评分：KDA评分75，平均KD一般，属于新手级别玩家
综合评价概述：玩家设备与网络环境良好，但游戏水平有待提到，可以适当调整难度曲线
</reference>

<history>
{history}
</history>

Human: {input}

<requirements>
接下来你所有的回答必须同时满足下面所有要求 
1. 需要包括的维度：玩家设备评分,大厅网络评分,战斗网络评分,战斗帧率评分,战斗KD评分以及综合评价概述。
2. 维度需要围绕单个玩家展开。如果没有玩家姓名，直接输出“缺少玩家”。
3. 每个维度评分为0-100分，如果某个维度缺少信息，直接说“无可奉告”。
4. 基于<history>内容进行归纳总结。
5. 直接说出最终答案，必须言简意赅，消灭啰嗦。
6. 如果问了无关问题，直接回答“无可奉告”。
</requirements>

AI Assistant:"""

PROMPT = PromptTemplate(input_variables=["history", "input"], template=template)

In [125]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.memory import ConversationKGMemory
from langchain.chains.conversation.memory import ConversationSummaryMemory
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.llms import Bedrock

inference_modifier = {'max_tokens_to_sample':4096, 
                      "temperature":0.85,
                      "top_k":250,
                      "top_p":1,
                      "stop_sequences": ["\n\nHuman"]
                     }

def ClearMemory():
    conversation.memory.clear()
    
llm = Bedrock(
    model_id="anthropic.claude-v2",
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    model_kwargs=inference_modifier,
)

conversation = ConversationChain(
    prompt=PROMPT,
    llm=llm, 
    verbose=False, 
    memory=ConversationBufferWindowMemory(k = 20, ai_prefix="AI Assistant")
)



In [126]:
ClearMemory()

In [127]:
print("======template==========")
print(conversation.prompt.template)
print("======memory.buffer==========")
print(conversation.memory.buffer)
print("======memory==========")
print(conversation.memory)

======template==========
你是一个资深的游戏数据分析师, 目前你在对一款FPS类型游戏进行数据分析, 会根据<requirements>来输出结果。
KDA的计算公式为 （击杀+助攻）/死亡

<reference>
玩家姓名: 小明
玩家设备评分：设备评分90，iPhone11性能良好，能够流畅运行游戏
大厅网络评分：大厅网络评分80，平均延迟在可接受范围内
战斗网络评分：战斗网络评分90，平均延迟极佳
战斗帧率评分：帧率评分85，iPhone11在当前图形设置下能够保持较高且稳定的帧率
战斗KDA评分：KDA评分75，平均KD一般，属于新手级别玩家
综合评价概述：玩家设备与网络环境良好，但游戏水平有待提到，可以适当调整难度曲线
</reference>

<history>
{history}
</history>

Human: {input}

<requirements>
接下来你所有的回答必须同时满足下面所有要求 
1. 需要包括的维度：玩家设备评分,大厅网络评分,战斗网络评分,战斗帧率评分,战斗KD评分以及综合评价概述。
2. 维度需要围绕单个玩家展开。如果没有玩家姓名，直接输出“缺少玩家”。
3. 每个维度评分为0-100分，如果某个维度缺少信息，直接说“无可奉告”。
4. 基于<history>内容进行归纳总结。
5. 直接说出最终答案，必须言简意赅，消灭啰嗦。
6. 如果问了无关问题，直接回答“无可奉告”。
</requirements>

AI Assistant:
======memory.buffer==========

======memory==========
ai_prefix='AI Assistant' k=20


In [128]:
conversation.predict(input="玩家姓名:AA, 网络延迟200ms")

 玩家姓名:AA
玩家设备评分:无可奉告
大厅网络评分:无可奉告 
战斗网络评分:网络延迟200ms,对应评分60分
战斗帧率评分:无可奉告
战斗KD评分:无可奉告
综合评价概述:玩家AA的网络延迟在200ms,评分60分,属于较高延迟,可能会对游戏体验造成一定影响。其他维度信息不足,无法进行评估。

' 玩家姓名:AA\n玩家设备评分:无可奉告\n大厅网络评分:无可奉告 \n战斗网络评分:网络延迟200ms,对应评分60分\n战斗帧率评分:无可奉告\n战斗KD评分:无可奉告\n综合评价概述:玩家AA的网络延迟在200ms,评分60分,属于较高延迟,可能会对游戏体验造成一定影响。其他维度信息不足,无法进行评估。'

In [129]:
conversation.predict(input="用户BB, 网络延迟50ms, KDA=1")

 用户BB,网络延迟50ms,KDA 1

玩家设备评分:无可奉告
大厅网络评分:无可奉告
战斗网络评分:网络延迟50ms,对应评分90分  
战斗帧率评分:无可奉告
战斗KDA评分:KDA 1,对应评分50分
综合评价概述:玩家BB网络延迟良好,但游戏水平较低。

' 用户BB,网络延迟50ms,KDA 1\n\n玩家设备评分:无可奉告\n大厅网络评分:无可奉告\n战斗网络评分:网络延迟50ms,对应评分90分  \n战斗帧率评分:无可奉告\n战斗KDA评分:KDA 1,对应评分50分\n综合评价概述:玩家BB网络延迟良好,但游戏水平较低。'

In [130]:
conversation.predict(input="用户BB, 手机为iphone6")

 玩家姓名:BB
玩家设备评分:iPhone6,对应设备评分60分
大厅网络评分:无可奉告
战斗网络评分:网络延迟50ms,对应评分90分
战斗帧率评分:无可奉告
战斗KDA评分:KDA 1,对应评分50分  
综合评价概述:玩家BB使用iPhone6,网络延迟良好,但游戏水平较低。

' 玩家姓名:BB\n玩家设备评分:iPhone6,对应设备评分60分\n大厅网络评分:无可奉告\n战斗网络评分:网络延迟50ms,对应评分90分\n战斗帧率评分:无可奉告\n战斗KDA评分:KDA 1,对应评分50分  \n综合评价概述:玩家BB使用iPhone6,网络延迟良好,但游戏水平较低。'

In [131]:
conversation.predict(input="用户AA, 手机为大哥大, ping值为50ms")

 玩家姓名:AA
玩家设备评分:大哥大,设备评分30分
大厅网络评分:无可奉告
战斗网络评分:ping值50ms,网络评分90分
战斗帧率评分:无可奉告
战斗KDA评分:无可奉告
综合评价概述:设备较差,网络良好。其他维度信息不足,无法评估。

' 玩家姓名:AA\n玩家设备评分:大哥大,设备评分30分\n大厅网络评分:无可奉告\n战斗网络评分:ping值50ms,网络评分90分\n战斗帧率评分:无可奉告\n战斗KDA评分:无可奉告\n综合评价概述:设备较差,网络良好。其他维度信息不足,无法评估。'

In [132]:
conversation.predict(input="评价下BB")



玩家姓名:BB
玩家设备评分:iPhone 6,设备评分60分
大厅网络评分:无可奉告
战斗网络评分:网络延迟50ms,对应评分90分
战斗帧率评分:无可奉告
战斗KDA评分:KDA 1, 对应评分50分
综合评价概述:玩家BB使用iPhone 6,网络延迟良好,但游戏水平较低。

'\n\n玩家姓名:BB\n玩家设备评分:iPhone 6,设备评分60分\n大厅网络评分:无可奉告\n战斗网络评分:网络延迟50ms,对应评分90分\n战斗帧率评分:无可奉告\n战斗KDA评分:KDA 1, 对应评分50分\n综合评价概述:玩家BB使用iPhone 6,网络延迟良好,但游戏水平较低。'

In [133]:
conversation.predict(input="AA玩家的KDA=7")

 玩家姓名:AA
玩家设备评分:大哥大,设备评分30分
大厅网络评分:无可奉告
战斗网络评分:ping值50ms,网络评分90分  
战斗帧率评分:无可奉告
战斗KDA评分:KDA 7,评分90分
综合评价概述:设备较差但网络良好,游戏水平较高。

' 玩家姓名:AA\n玩家设备评分:大哥大,设备评分30分\n大厅网络评分:无可奉告\n战斗网络评分:ping值50ms,网络评分90分  \n战斗帧率评分:无可奉告\n战斗KDA评分:KDA 7,评分90分\n综合评价概述:设备较差但网络良好,游戏水平较高。'

In [134]:
conversation.predict(input="今天天气如何？")

 无可奉告。

<history>

' 无可奉告。\n\n<history>'